In [1]:
import os
import json

import torch
import torch.nn as nn

from ninpy.datasets import BurstDataset

In [ ]:
class PascalObject(BurstDataset):
    def __init__(self):
        super().__init__()
    
    def prepare(self):
        return


In [ ]:
def cvt_xml2csv(xml_dir):
    """Parse xml file."""
    return

def prepare_labels():
    return



In [4]:
list(map(lambda x: x**2, [1, 2, 3]))

[1, 4, 9]